In [21]:
import argparse
import pandas as pd
import sys

# allow load_data funcs
sys.path.append("../data/")
from load_data import load_ucimlrepo

In [22]:
data = load_ucimlrepo().data
X = data.features
y = data.targets


In [16]:
del data # save space

In [4]:
print(X.keys())
print(y.keys())

Index(['AAGE', 'ACLSWKR', 'ADTINK', 'ADTOCC', 'AHGA', 'AHSCOL', 'AMARITL',
       'AMJIND', 'AMJOCC', 'ARACE', 'AREORGN', 'ASEX', 'AUNMEM', 'AUNTYPE',
       'AWKSTAT', 'CAPGAIN', 'GAPLOSS', 'DIVVAL', 'FILESTAT', 'GRINREG',
       'GRINST', 'HHDFMX', 'HHDREL', 'MARSUPWRT', 'MIGMTR1', 'MIGMTR3',
       'MIGMTR4', 'MIGSAME', 'MIGSUN', 'NOEMP', 'PARENT', 'PEFNTVTY',
       'PEMNTVTY', 'PENATVTY', 'PRCITSHP', 'SEOTR', 'VETQVA', 'VETYN',
       'WKSWORK', 'AHRSPAY', 'year'],
      dtype='object')
Index(['income'], dtype='object')


# prep data

In [23]:
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder

In [ ]:
X.query()

In [24]:
ordenc = OrdinalEncoder()
for col in X.columns:
    if X[col].nunique() > 52 or col == 'NOEMP':
        X[col] = ordenc.fit_transform(X[col])

ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

In [18]:
def prepare_inputs_dual(X_train, X_test):
    # https://machinelearningmastery.com/feature-selection-with-categorical-data/
	oe = OrdinalEncoder()
	oe.fit(X_train)
	X_train_enc = oe.transform(X_train)
	X_test_enc = oe.transform(X_test)
	return X_train_enc, X_test_enc

# prepare target
def prepare_targets_dual(y_train, y_test):
    # https://machinelearningmastery.com/feature-selection-with-categorical-data/
	le = LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	return y_train_enc, y_test_enc

def prepare_inputs_single(X):
    # use for whooooole training dataset
    # https://machinelearningmastery.com/feature-selection-with-categorical-data/
	oe = OrdinalEncoder()
	oe.fit(X)
	X_enc = oe.transform(X)
	return X_enc

# prepare target
def prepare_targets_single(y):
    # use for whooooooole label of training dataset
    # https://machinelearningmastery.com/feature-selection-with-categorical-data/
	le = LabelEncoder()
	le.fit(t)
	y_enc = le.transform(y)
	return y_enc

In [ ]:
print("loaded. encoding data...")
# prep data
X_enc = prepare_inputs_single(X)
y_enc = prepare_targets_single(y)
print("encoded.")

NameError: name 'y' is not defined

# Random Forest

In [6]:
# random forest imports
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score ,RepeatedStratifiedKFold,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit

# metrics
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, ConfusionMatrixDisplay
from functools import partial # beacause I want to send in a param to f1 to cross val
import seaborn as sns

In [6]:
from sklearn.model_selection import RandomizedSearchCV
#     "criterion":['gini','entropy'],
params = {
    "n_estimators":[50,100,500,1000],
    "max_features":["log2","sqrt",None],
    "min_samples_leaf":[1,2,3,4],
}
oob = partial(f1_score,average='weighted') # because accuracy by default for random forest
n_jobs = -1



In [7]:
rf = RandomForestClassifier(oob_score=oob,n_jobs=n_jobs,class_weight='balanced_subsample')

In [10]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = params, n_iter = 20, cv = 3, verbose=4, random_state=42, n_jobs = -1)
rf_random.fit(X, y)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


ValueError: 
All the 60 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.11/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/sklearn/ensemble/_forest.py", line 363, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/sklearn/base.py", line 650, in _validate_data
    X, y = check_X_y(X, y, **check_para

[CV 2/3] END max_features=sqrt, min_samples_leaf=3, n_estimators=100;, score=nan total time=   0.1s
[CV 3/3] END max_features=sqrt, min_samples_leaf=3, n_estimators=100;, score=nan total time=   0.1s
[CV 1/3] END max_features=sqrt, min_samples_leaf=3, n_estimators=1000;, score=nan total time=   0.2s
[CV 3/3] END max_features=log2, min_samples_leaf=3, n_estimators=50;, score=nan total time=   0.1s
[CV 1/3] END max_features=log2, min_samples_leaf=3, n_estimators=50;, score=nan total time=   0.1s
[CV 2/3] END max_features=sqrt, min_samples_leaf=3, n_estimators=1000;, score=nan total time=   0.1s
[CV 1/3] END max_features=log2, min_samples_leaf=1, n_estimators=1000;, score=nan total time=   0.1s
[CV 2/3] END max_features=log2, min_samples_leaf=3, n_estimators=50;, score=nan total time=   0.1s
[CV 3/3] END max_features=sqrt, min_samples_leaf=3, n_estimators=1000;, score=nan total time=   0.2s
[CV 2/3] END max_features=log2, min_samples_leaf=1, n_estimators=1000;, score=nan total time=   0.1